In [1]:
import numpy as np
import capytaine as cpt
import xarray as xr
from math import pi


mesh_file = "2112open.stl"
rotation_center = None
speeds = np.linspace(0.5, 1.3, 9)
wave_periods = np.linspace(0.5, 1.0, 21)
wave_direction = np.pi
g = 9.81
rho = 1000.0
 
boat_mesh = cpt.load_mesh(mesh_file, file_format="stl")
if rotation_center is None:
    rotation_center = boat_mesh.center_of_buoyancy

lid_mesh = boat_mesh.generate_lid()
boat = cpt.FloatingBody(
    mesh=boat_mesh.immersed_part(),
    lid_mesh=lid_mesh,
    dofs=cpt.rigid_body_dofs(rotation_center=boat_mesh.center_of_buoyancy),
    center_of_mass=boat_mesh.center_of_buoyancy,
    name="pirate ship" 
 )
boat.mass = boat.immersed_part().volume * rho
boat.inertia_matrix = boat.compute_rigid_body_inertia()
boat.hydrostatic_stiffness = boat.immersed_part().compute_hydrostatic_stiffness(rho=rho, g=g)

solver = cpt.BEMSolver()
omega_abs = 2 * pi / wave_periods
k_abs = omega_abs ** 2 / g

rao_data = {}
for U in speeds:
    ω = omega_abs
    k = k_abs
    ds_problem = xr.Dataset(coords={
        "omega": ω,
        "wave_direction": [wave_direction],
        "radiating_dof": list(boat.dofs.keys()),
        "water_depth": [np.inf], 
        "rho": [rho],
        "g": [g],
        "forward_speed": [U]
    })
    results_ds = solver.fill_dataset(ds_problem, boat)
    rao_ds = cpt.post_pro.rao(results_ds)
    rao_data[U] = {
        "omega_abs": ω,
        "H3": rao_ds.sel(radiating_dof="Heave", forward_speed=U, wave_direction=wave_direction).values,
        "H5": rao_ds.sel(radiating_dof="Pitch", forward_speed=U, wave_direction=wave_direction).values,
        "k": k
    }

steepness = 1/40.0
results = {}

for U, data in rao_data.items():
    ω_abs_ = data["omega_abs"]
    H3 = data["H3"]
    H5 = data["H5"]
    k = data["k"]
    x, y, z0 = -0.5, 0.0, 0.01
    Hsz = H3 - x * H5
    Hζ = np.exp(-1j * k * x)
    HR = Hζ - Hsz
    A_wave = steepness / k
    amp_rel = np.abs(HR) * A_wave
    C = np.clip(z0 / amp_rel, -1.0, 1.0)
    theta = np.arccos(C) 
    frac_wet = theta / pi
    results[U] = {
        "omega_abs": ω_abs_,
        "A_wave": A_wave,
        "amp_rel": amp_rel,
        "frac_wet": frac_wet
    }

for U, res in results.items():
    print(f"\nSpeed U = {U:.2f} m/s (steepness = 1/40)")
    for ω, Aw, amp, fw in zip(res["omega_abs"], res["A_wave"], res["amp_rel"], res["frac_wet"]):
        print(f" ω_abs = {ω:.2f} rad/s → A_wave = {Aw:.3f} m, |R| = {amp:.3f} m, wetted frac = {fw:.2f}")

def wetted_area_from_submersion(h):
    h_pos = np.clip(h, 0.0, None)
    coeffs = [
        -699.844799,
         176.787381, 
         -18.242259,
           1.16326744,
           0.0484737259,
          -1.94070903e-05
    ]
    area_half = np.polyval(coeffs, h_pos)
    return 2.0 * np.clip(area_half, 0.0, None)

def mean_wetted_area_cycle(z0, amp_rel, n_pts=200):
    thetas = np.linspace(0, 2*np.pi, n_pts, endpoint=False)
    H = z0 + amp_rel[:, None] * np.cos(thetas)
    H = np.clip(H, 0.0, None)
    A = wetted_area_from_submersion(H)
    return A.mean(axis=1)

area_results = {U: {"mean": [], "max": [], "min": []} for U in speeds}
for U, res in results.items(): 
    z0 = 0.01
    amp_rel = res["amp_rel"]
    sub_max = z0 + amp_rel
    sub_min = np.clip(z0 - amp_rel, 0.0, None)
    area_results[U]["mean"] = mean_wetted_area_cycle(z0, amp_rel)
    area_results[U]["max"] = wetted_area_from_submersion(sub_max)
    area_results[U]["min"] = wetted_area_from_submersion(sub_min)





U_arr = np.array(speeds)
T_arr = wave_periods
Ug, Tg = np.meshgrid(U_arr, T_arr, indexing="ij")
mean_grid = np.vstack([area_results[U]["mean"] for U in speeds])
max_grid = np.vstack([area_results[U]["max"] for U in speeds])
min_grid = np.vstack([area_results[U]["min"] for U in speeds])
fig, axs = plt.subplots(1, 3, figsize=(15, 4), sharey=True)
for ax, Z, title in zip(axs, (mean_grid, max_grid, min_grid), ("Mean wetted area", "Max wetted area", "Min wetted area")):
    pcm = ax.pcolormesh(Tg, Ug, Z, shading="auto")
    fig.colorbar(pcm, ax=ax)
    ax.set_xlabel("Wave period [s]")
    ax.set_title(title)
axs[0].set_ylabel("Speed [m/s]")
plt.tight_layout()
fig.savefig("wetted_area_grids.png", dpi=300)
plt.show()


Output()

KeyboardInterrupt: 

In [4]:
# Save each panel separately
titles = ["mean", "max", "min"]
grids = [mean_grid, max_grid, min_grid]

for grid, name in zip(grids, titles):
    fig, ax = plt.subplots(figsize=(5, 4))
    pcm = ax.pcolormesh(Tg, Ug, grid, shading="auto")
    fig.colorbar(pcm, ax=ax)
    ax.set_title(f"{name.capitalize()} wetted area")
    ax.set_xlabel("Wave period [s]")
    ax.set_ylabel("Speed [m/s]")
    fig.tight_layout()
    fig.savefig(f"wetted_area_{name}.png", dpi=300)
    plt.close(fig)

In [5]:
# Save each panel separately with Speed on x‐axis and Period on y‐axis
titles = ["mean", "max", "min"]
grids = [mean_grid, max_grid, min_grid]

for grid, name in zip(grids, titles):
    fig, ax = plt.subplots(figsize=(5, 4))
    # swap the X/Y arguments
    pcm = ax.pcolormesh(Ug, Tg, grid, shading="auto")
    fig.colorbar(pcm, ax=ax)
    ax.set_title(f"{name.capitalize()} wetted area")
    ax.set_xlabel("Speed [m/s]")
    ax.set_ylabel("Wave period [s]")
    fig.tight_layout()
    fig.savefig(f"wetted_area_{name}.png", dpi=300)
    plt.close(fig)

In [6]:
titles = ["mean", "max", "min"]
grids = [mean_grid, max_grid, min_grid]

for grid, name in zip(grids, titles):
    fig, ax = plt.subplots(figsize=(5, 4))
    pcm = ax.pcolormesh(Ug, Tg, grid, shading="auto")
    fig.colorbar(pcm, ax=ax)
    ax.set_title(f"{name.capitalize()} wetted area")
    ax.set_xlabel("Speed [m/s]")
    ax.set_ylabel("Wave period [s]")
    # invert y-axis so smallest period is at the top
    ax.invert_yaxis()
    fig.tight_layout()
    fig.savefig(f"wetted_area_{name}.png", dpi=300)
    plt.close(fig)

In [ ]:
def wetted_area_from_submersion(h):
    """
    Returns wetted area [m²] for a given waterline height h [m].
    Uses the pre-computed 5th-degree fit.
    """
    h_pos = np.clip(h, 0.0, None)
    # polynomial coefficients, highest power first
    coeffs = [
        -699.844799,   # h^5
         176.787381,   # h^4
         -18.242259,   # h^3
           1.16326744, # h^2
           0.0484737259, # h^1
          -1.94070903e-05  # constant
    ]
    # Evaluate polynomial at h
    area_half = np.polyval(coeffs, h_pos)

    return 2.0 * np.clip(area_half, 0.0, None)
print(wetted_area_from_submersion(0.01))  
print(wetted_area_from_submersion(0.02))   # Test 
print(wetted_area_from_submersion(0.00))   # Test 


0.0011302250860602
0.0025909656186064
0.0


In [9]:
print(boat.inertia_matrix)

<xarray.DataArray 'inertia_matrix' (influenced_dof: 6, radiating_dof: 6)> Size: 288B
array([[ 1.41639131e+01,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00, -0.00000000e+00],
       [ 0.00000000e+00,  1.41639131e+01,  0.00000000e+00,
        -0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  1.41639131e+01,
         0.00000000e+00, -0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00, -0.00000000e+00,  0.00000000e+00,
         6.14428404e-02,  1.60662266e-02,  1.56877564e-02],
       [ 0.00000000e+00,  0.00000000e+00, -0.00000000e+00,
         1.60662266e-02,  6.41172366e-01, -4.89572489e-04],
       [-0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.56877564e-02, -4.89572489e-04,  6.69581270e-01]])
Coordinates:
  * influenced_dof  (influenced_dof) <U5 120B 'Surge' 'Sway' ... 'Pitch' 'Yaw'
  * radiating_dof   (radiating_dof) <U5 120B 'Surge' 'Sway' ... 'Pitch' 'Yaw'
